In [1]:
import json 
import numpy as np
import pandas as pd
from nltk_utils import tokenize, stem, bag_of_words

import tensorflow as tf

In [2]:
# import nltk
# nltk.download("punkt")

To activate speech, run the cell below

# Train

In [3]:
ignore_words = ["?", "i", "it", "a", "!", "me", "you", "'s", "does", "of", "do", "my", "are", "you"]

In [4]:
# get tags and set of all words from training data
# xy contains sample prompt and its expected output category

with open("intents.json", "r") as f:
    intents = json.load(f)

tags = []
all_words = []
xy = []

for intent in intents["intents"]:
    tag = intent["tag"]
    tags.append(tag)
    
    for pattern in intent["patterns"]:
        
        words = tokenize(pattern)
        l1 = [stem(w) for w in words if w.lower() not in ignore_words]
        
        all_words.extend(l1)
        xy.append((l1, tag))

all_words = sorted(set(all_words))
np.random.shuffle(xy)

In [5]:
print(tags)
print(all_words)
print(xy)

['unknown', 'greeting', 'goodbye', 'thanks', 'items', 'payments', 'delivery', 'funny']
['appreci', 'avail', 'buy', 'bye', 'can', 'card', 'cash', 'credit', 'deliv', 'deliveri', 'expect', 'funni', 'goodby', 'have', 'hello', 'help', 'hey', 'hi', 'how', 'is', 'item', 'joke', 'know', 'later', 'laugh', 'lot', 'make', 'okay', 'on', 'packag', 'pay', 'payment', 'paypal', 'product', 'realli', 'reciev', 'sell', 'ship', 'should', 'someth', 'thank', 'that', 'the', 'time', 'up', 'use', 'what', 'when', 'will']
[(['sell'], 'items'), (['what', 'is', 'the', 'expect', 'deliveri', 'time'], 'delivery'), (['can', 'use', 'paypal'], 'payments'), (['hello'], 'greeting'), (['how', 'can', 'make', 'payment'], 'payments'), (['thank', 'lot'], 'thanks'), (['when', 'will', 'reciev', 'packag'], 'delivery'), (['later'], 'goodbye'), (['thank'], 'thanks'), (['funni', 'joke'], 'funny'), (['joke'], 'funny'), (['know', 'joke'], 'funny'), (['avail'], 'items'), (['hey'], 'greeting'), (['what', 'up'], 'greeting'), (['item'], '

In [6]:
df = pd.DataFrame(xy)
df.head()

,0,1
0,[sell],items
1,"[what, is, the, expect, deliveri, time]",delivery
2,"[can, use, paypal]",payments
3,[hello],greeting
4,"[how, can, make, payment]",payments


In [7]:
df[df[1] == "delivery"]

,0,1
1,"[what, is, the, expect, deliveri, time]",delivery
6,"[when, will, reciev, packag]",delivery
20,[deliv],delivery
26,[ship],delivery


In [8]:
# Prepare training set

X_train = []
y_train = []

for sentence, tag in xy:
    bag = bag_of_words(sentence, all_words)
    X_train.append(bag)

    label = bag_of_words(tag, tags)
    y_train.append(label)

X_train = np.array(X_train)
y_train = np.array(y_train)

In [9]:
X_train

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [10]:
y_train

array([[0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 

# Model

In [11]:
ann = tf.keras.models.Sequential()

ann.add(tf.keras.layers.Dense(units=16, input_shape = np.shape(all_words), activation='relu'))
ann.add(tf.keras.layers.Dense(units=16, activation='relu'))

ann.add(tf.keras.layers.Dense(units=len(tags) , activation='softmax'))

ann.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics=['accuracy']) 

d:\AI\Projects\Basic Chatbot\venv\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [12]:
ann.fit(X_train, y_train, batch_size=6, epochs = 200)

Epoch 1/200
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.1576 - loss: 2.0632  
Epoch 2/200
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 0s/step - accuracy: 0.2094 - loss: 2.0560  
Epoch 3/200
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 0s/step - accuracy: 0.2586 - loss: 2.0314  
Epoch 4/200
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.2189 - loss: 2.0310
Epoch 5/200
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 0s/step - accuracy: 0.2904 - loss: 2.0016  
Epoch 6/200
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 0s/step - accuracy: 0.2626 - loss: 1.9894  
Epoch 7/200
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.2102 - loss: 2.0024 
Epoch 8/200
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 0s/step - accuracy: 0.1924 - loss: 2.0189  
Epoch 9/200
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.2105 - loss: 1.9625 
Epoch 10/200
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 0s/step - accuracy: 0.1926 - loss: 1.9876  
Epoch 11/200
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 0s/step - accuracy: 0.2663 - loss: 1.9415  
Epoch 12/200
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 0s/step - accuracy: 0.2683 - loss

In [13]:
# Tag index of input sentence

# input needs to be tokenized, stemmed and vectorized
input_1 = "hello"

# ann.predict(bag_of_words(input_1, all_words).reshape(1, -1))
np.argmax(ann.predict(bag_of_words(input_1, all_words).reshape(1, -1)))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step


1

# Response

In [14]:
# Respond based on given set of training responses

def respond(sentence):

    '''
    input : 
        sentence : input prompt from user

    output : 
        response : text from given set of responses in training data
        tag :      tag of input prompt
    '''

    words = tokenize(sentence)
    sentence = [stem(w) for w in words if w not in ignore_words]
    bag = bag_of_words(sentence, all_words).reshape(1, -1)
    
    probs = ann.predict(bag)

    if max(probs[0]) < 0.3 :
        max_idx = 0
    else :
        max_idx = np.argmax(probs)
    response = np.random.choice(intents['intents'][max_idx]['responses'])
    tag = intents['intents'][max_idx]['tag']

    return response, tag


In [15]:
input_2 = "how can i pay ?"

respond(input_2)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step


('We accept VISA, Mastercard and Paypal', 'payments')

# Speech

In [16]:
import speech_recognition as sr 
import pyttsx3

engine = pyttsx3.init()

In [17]:
# list of languages available in your windows

for voice in engine.getProperty('voices'):
    print(voice.id)

HKEY_LOCAL_MACHINE\SOFTWARE\Microsoft\Speech\Voices\Tokens\TTS_MS_EN-US_DAVID_11.0
HKEY_LOCAL_MACHINE\SOFTWARE\Microsoft\Speech\Voices\Tokens\MSTTS_V110_hiIN_KalpanaM
HKEY_LOCAL_MACHINE\SOFTWARE\Microsoft\Speech\Voices\Tokens\MSTTS_V110_hiIN_HemantM
HKEY_LOCAL_MACHINE\SOFTWARE\Microsoft\Speech\Voices\Tokens\TTS_MS_EN-US_ZIRA_11.0


In [18]:
# Assigning language and voice

lang = 'english'
voc = engine.getProperty('voices')[0].id

engine.setProperty('voice', voc)

In [19]:
# Speech to text

def get_transcript(lang):

    ''' 
    input : 
        lang : language of voice input
        
    output : transcript, bool
        if no error, returns transcript of given voice input and true.
        else, returns error message and false.
    '''

    recognizer = sr.Recognizer()
    audio = ' '
    transcript = ' '

    print("\nListening ......\n")
    
    with sr.Microphone() as source:
        audio = recognizer.listen(source)

    try:
        transcript = recognizer.recognize_google(audio, language=lang)
        print(transcript)
        return  transcript, True
    
    except sr.UnknownValueError:
        print("Could not understand audio")
        return "Could not understand audio", False
    
    except sr.RequestError as e:
        print("Could not request results")
        return "Could not request results", False
 

In [20]:
# Text to speech

def speak(response):
    
    '''
    input : response 
        text to be converted to speech
    '''

    engine.say(response)
    engine.runAndWait()


# Main

In [22]:
while True :

    text, correct = get_transcript(lang)

    if correct :

        print("You : ", text)
        response, tag = respond(text)

        print("Chatbot : ", response)
        speak(response)

        if tag == 'goodbye' :
            break
    else :

        speak(text)


Listening ......

hello
You :  hello
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
Chatbot :  Hi, how can I assist you today??

Listening ......

when will I receive my package
You :  when will I receive my package
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
Chatbot :  Shipping takes 2-3 days

Listening ......

do you have cash on delivery
You :  do you have cash on delivery
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
Chatbot :  We accept VISA, Mastercard and Paypal

Listening ......

thank you
You :  thank you
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
Chatbot :  My pleasure

Listening ......

bye
You :  bye
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
Chatbot :  See you later, thanks for chatting
